# Überschrift

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# 📥 Daten laden
train_df = pd.read_csv("..//data//train.csv")
test_df = pd.read_csv("..//data//test.csv")

# 🎯 Zielvariable
y = train_df["Survived"]

# 🧹 Features vorbereiten
X = train_df.drop(["Survived", "Name", "Ticket", "Cabin", "PassengerId"], axis=1)

X["Sex"] = X["Sex"].map({"male": 0, "female": 1})
X["Embarked"].fillna("S", inplace=True)
X = pd.get_dummies(X, columns=["Embarked"], drop_first=True)
X["Age"].fillna(X["Age"].median(), inplace=True)

# Titel extrahieren 
titles = train_df["Name"].str.extract(r" ([A-Za-z]+)\.", expand=False)
# Weniger häufig vorkommende Titel werden zur Gruppe "Rare" zusammengefasst.
titles = titles.replace(
    ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
    'Rare'
)
# Manchmal tauchen gleiche Rollen in leicht anderer Schreibweise auf, Mlle ist französisch für Miss und Mme für Mrs.
titles = titles.replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
# One-Hot-Encoding oder Label-Encoding (hier Label als Beispiel)
X["Title"] = LabelEncoder().fit_transform(titles)

# Familiengröße und alleinstehend
# Die Anzahl der Geschwister/Ehepartner (SibSp) und Eltern/Kinder (Parch) wird addiert, um die Familiengröße zu berechnen.
X["FamilySize"] = train_df["SibSp"] + train_df["Parch"] + 1
X["IsAlone"] = (X["FamilySize"] == 1).astype(int)  # 1 = allein, 0 = nicht allein



# ✂️ Trainings-/Validierungsdaten splitten
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 🧠 Modell trainieren
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# ✅ Evaluation
y_pred = model.predict(X_val)
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))



C:\Users\timgr_\AppData\Local\Temp\ipykernel_14440\3125251861.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X["Embarked"].fillna("S", inplace=True)
C:\Users\timgr_\AppData\Local\Temp\ipykernel_14440\3125251861.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

[[90 15]
 [20 54]]
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       105
           1       0.78      0.73      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179



Bei den Koeffizienten sieht man, dass Frauen starke überlebenschancen haben mit 2,59 log odd = 13.34 odds ratio = Wenn das Geschlecht von männlich (0) auf weiblich (1) wechselt, dann steigen die Überlebens-Odds um den Faktor 13.34.
Bei Pclass -0,93 = bedeutet, dass Für jede Erhöhung der Klasse (z. B. von 1 → 2 oder 2 → 3) = schlechtere Klasse = sinken die Überlebens-Odds um etwa 61 % (da e^-0,93 ≈ 0.39).

In [8]:
# 📊 Koeffizienten anzeigen (Einfluss der Features)
coefficients = pd.Series(model.coef_[0], index=X.columns)
print("\nKoeffizienten (log-Odds):")
print(coefficients.sort_values(ascending=False))

# 🔁 Odds Ratios (besser interpretierbar als Wahrscheinlichkeitsänderung)
odds_ratios = np.exp(coefficients)
print("\nOdds Ratios (e^Koeffizient):")
print(odds_ratios.sort_values(ascending=False))


Koeffizienten (log-Odds):
Sex           2.526754
Fare          0.002159
Parch        -0.003948
Age          -0.025859
Embarked_Q   -0.095095
Title        -0.122959
SibSp        -0.227750
FamilySize   -0.248710
Embarked_S   -0.342891
IsAlone      -0.701901
Pclass       -0.887476
dtype: float64

Odds Ratios (e^Koeffizient):
Sex           12.512830
Fare           1.002162
Parch          0.996060
Age            0.974473
Embarked_Q     0.909287
Title          0.884300
SibSp          0.796323
FamilySize     0.779806
Embarked_S     0.709715
IsAlone        0.495642
Pclass         0.411693
dtype: float64
